In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import cv2
import os

# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Some modules to display an animation using imageio.
import imageio
from IPython.display import HTML, display

In [2]:
import tensorflow as tf

# 设置本地 TFLite 模型文件的路径
model_path = r"C:\Users\ASUS\Desktop\school\project\TFLiteMovenet\lite-model_movenet_singlepose_lightning_3.tflite"  # 替换为你本地模型文件的路径

# 初始化 TFLite 解释器
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# 获取输入和输出张量的索引
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 定义输入图像的大小
input_size = 192  # 根据你的模型配置进行调整

def movenet(input_image):
    """运行输入图像上的检测。

    Args:
      input_image: 一个 [1, height, width, 3] 张量，表示输入图像像素。请注意，高度/宽度应该已经被调整大小并与模型的期望输入分辨率匹配，然后再传递给此函数。

    Returns:
      一个 [1, 1, 17, 3] 的浮点 numpy 数组，表示预测的关键点坐标和分数。
    """
    # TF Lite 格式期望 tensor 类型为 float32。
    input_image = tf.cast(input_image, dtype=tf.float32)
    
    # 设置输入张量
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    
    # 运行模型推断
    interpreter.invoke()
    
    # 获取模型输出
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    return keypoints_with_scores


In [3]:
# 图像文件夹路径
image_folder=r"C:\Users\ASUS\Desktop\school\project\poseture\allPoseture"
output_folder = r"C:\Users\ASUS\Desktop\school\project\poseture\Keypoints"

# 遍历图像文件夹中的每个图像文件

# Iterate over each image file in the image folder
for filename in os.listdir(image_folder):
    if filename.endswith('.jpg') or filename.endswith('.JPG'):
        # Read the image
        image_path = os.path.join(image_folder, filename)
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image)
        
        # Resize and pad the image
        input_image = tf.expand_dims(image, axis=0)
        input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

        # Run model inference
        keypoints_with_scores = movenet(input_image)
        
        #存節點
        output_path = os.path.join(output_folder, filename.replace('.jpg', '.npy').replace('.JPG', '.npy'))
        np.save(output_path, keypoints_with_scores)


In [ ]:
keypoints = np.load(r"C:\Users\ASUS\Desktop\school\project\poseture\Keypoints\falldown336.npy")
print(keypoints)
print(keypoints.shape)